In [ ]:
!pip install -q simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import tarfile
import pandas as pd
import textwrap
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score
seed = 42

In [ ]:
df = pd.read_csv("cleaned-testDataSet.csv") #, encoding_errors='replace')
print(df.size)
df

156612


,Unnamed: 0,Content,Author
0,0,भाग पहिला,Ranjeet Desai
1,1,,Ranjeet Desai
2,2,१,Ranjeet Desai
3,3,शिवनेरी पायथ्याचे जुन्नर गाव पश्चिमेकडे झुकले...,Ranjeet Desai
4,4,'आबासाहेब; मासाहेब आल्या.' शंभूराजे म्हणाले .,Ranjeet Desai
...,...,...,...
52199,52199,NaN,Sane Guruji
52200,52200,"""मित्रांनो! अशी माझी आई होती. जगाच्या बाजारात ...",Sane Guruji
52201,52201,NaN,Sane Guruji
52202,52202,NaN,Sane Guruji


In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['Content'].str.len().value_counts().sort_index().head()

1    7377
3     187
4     351
5      81
6      59
Name: Content, dtype: int64

In [ ]:
df = df[df['Content'].str.len() > 1]

In [ ]:
df

,Content,Author
0,भाग पहिला,Ranjeet Desai
2,१,Ranjeet Desai
3,शिवनेरी पायथ्याचे जुन्नर गाव पश्चिमेकडे झुकले...,Ranjeet Desai
4,'आबासाहेब; मासाहेब आल्या.' शंभूराजे म्हणाले .,Ranjeet Desai
5,'कुठं?' मान वर करीत शहाजीराजे विचारते झाले.,Ranjeet Desai
...,...,...
52192,माझ्या आईवर बायामाणसांचे प्रेम होतेच; परंतु पश...,Sane Guruji
52194,मथी नेहमी आईच्या भोवती भोवती असावयाची. आई शौचा...,Sane Guruji
52196,आईचे आजारपण वाढत चालले; तसतशी मथीही नीट खातपीत...,Sane Guruji
52198,ज्या दिवशी आई गेली; त्या दिवशी ती सारखी म्यांव...,Sane Guruji


In [ ]:
df = df.astype({'Content': str}).groupby(['Author'])['Content'].apply(' '.join).reset_index()
df

,Author,Content
0,Author,Content Content Content Content Content Content
1,Ranjeet Desai,भाग पहिला १ शिवनेरी पायथ्याचे जुन्नर गाव ...
2,Sane Guruji,पुष्कळ वेळा मनुष्याचा मोठेपणा त्याच्या आईबापां...
3,Shivaji Sawant,१. श्रीकृष्ण मलाही आज बोल...
4,V S Khandekar,भाग पहिला ययाति १ माझ...


In [ ]:
df.drop([2], inplace=True)

In [ ]:
df.drop([0], inplace=True)
df = df.reset_index(drop=True)
df

,Author,Content
0,Ranjeet Desai,भाग पहिला १ शिवनेरी पायथ्याचे जुन्नर गाव ...
1,Shivaji Sawant,१. श्रीकृष्ण मलाही आज बोल...
2,V S Khandekar,भाग पहिला ययाति १ माझ...


In [ ]:
df['Content'].str.len()

0    2438440
1    3561487
2    1215011
Name: Content, dtype: int64

In [ ]:
width = min(df['Content'].str.len())
df['Content'] = df['Content'].apply(lambda s: textwrap.shorten(s, width=width, placeholder=" "))
df

,Author,Content
0,Ranjeet Desai,भाग पहिला १ शिवनेरी पायथ्याचे जुन्नर गाव पश्चि...
1,Shivaji Sawant,१. श्रीकृष्ण मलाही आज बोललंच पाहिजे!! असं चमकू...
2,V S Khandekar,भाग पहिला ययाति १ माझी कहाणी मी का सांगत आहे? ...


In [ ]:
df['Content'].str.len()

0    1215004
1    1215006
2    1164101
Name: Content, dtype: int64

In [ ]:
labels = list(df['Author'].unique())
labels.sort()
print(labels)
mapper = {labels[i]: i for i in range(len(labels))}
print(mapper)

df.replace({'Author': mapper}, inplace=True)
df.astype({'Author': 'int'})
df.rename(columns={'Author': 'Labels'}, inplace=True)
df = df.iloc[:,[1, 0]]
df

['Ranjeet Desai', 'Shivaji Sawant', 'V S Khandekar']
{'Ranjeet Desai': 0, 'Shivaji Sawant': 1, 'V S Khandekar': 2}


,Content,Labels
0,भाग पहिला १ शिवनेरी पायथ्याचे जुन्नर गाव पश्चि...,0
1,१. श्रीकृष्ण मलाही आज बोललंच पाहिजे!! असं चमकू...,1
2,भाग पहिला ययाति १ माझी कहाणी मी का सांगत आहे? ...,2


In [ ]:
train_df = df.copy()
test_df = df.copy()

In [ ]:
train_df['Content'] = train_df['Content'].apply(lambda s: textwrap.wrap(s, width*0.8, placeholder=" "))

In [ ]:
splitted = pd.DataFrame(train_df["Content"].to_list(), columns=['train', 'test'])
splitted

,train,test
0,भाग पहिला १ शिवनेरी पायथ्याचे जुन्नर गाव पश्चि...,आजपासून हा मोती तुमचा. फार समजुतदार जनावर आहे....
1,१. श्रीकृष्ण मलाही आज बोललंच पाहिजे!! असं चमकू...,दर्शवी. यात त्याचं कृष्ण आणि अर्जुनप्रेमही व्य...
2,भाग पहिला ययाति १ माझी कहाणी मी का सांगत आहे? ...,कुबेर वकिलांचा माधव म्हणून मुलगा आहे कॉलेजात. ...


In [ ]:
splitted.apply(lambda s: s.str.len())

,train,test
0,972008,242994
1,972007,242997
2,972004,192096


In [ ]:
train_df['Content'] = splitted['train']
test_df['Content'] = splitted['test']

In [ ]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//80 >0:
        n = len(text1.split())//80
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:100]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*80:w*80 + 100]
            l_total.append(" ".join(l_parcial))
    return l_total

In [ ]:
train_df['Content'] = train_df['Content'].apply(get_split)
train_df = train_df.explode(column=['Content']).reset_index(drop=True)
test_df['Content'] = test_df['Content'].apply(get_split)
test_df = test_df.explode(column=['Content']).reset_index(drop=True)

In [ ]:
train_df

,Content,Labels
0,भाग पहिला १ शिवनेरी पायथ्याचे जुन्नर गाव पश्चि...,0
1,करीत शहाजीराजे विचारते झाले. शंभूबाळांनी बोट क...,0
2,ठायी ठायी थांबत आम्ही दौड करीत राहिलो; तर तुमच...,0
3,समर्थ आहोत; म्हणावं.' 'मी काय बोलणार यात?' जिज...,0
4,ही तकलीफ देत आहोत. त्याबद्दल आम्ही शरमिंदे आहो...,0
...,...,...
5488,पूर्वी. आता काळ बदलला; तेव्हा आपण अंधार-भोजन क...,2
5489,पाहुण्यांची व्यवस्था लावायला मी सुरुवात केली. ...,2
5490,आणा. आरसा आणा. इथं एक सुंदर कंदील ठेवा. छे! छे...,2
5491,किती वेळ बसायचं माणसानं?' पापासाहेबांकडले शिकव...,2


In [ ]:
test_df

,Content,Labels
0,आजपासून हा मोती तुमचा. फार समजुतदार जनावर आहे....,0
1,सांभाळा. सकाळ संध्याकाळ पागेत चक्कर टाका. त्या...,0
2,नेहमीप्रमाणे जिजाबाईंच्या महाली गेले. मासाहेबा...,0
3,शंभूबाळ. आपण बसू.' राजांच्या पाठोपाठ जिजाबाई श...,0
4,मदत करू नका; असा इशारा त्यांनी दिला आहे. पुणे;...,0
...,...,...
1254,क्रमाक्रमाने नाकेबंदी होऊ लागली; अशा वर्गात तो...,2
1255,हे सांगण्याचा माझा अधिकार नाही. 'समस्येचे स्वर...,2
1256,जो माणुसकीचा दिवा पिढ्यान्पिढ्या हातात धरला आह...,2
1257,स्थितीत माणुसकीचा आधार एकच आहे. तो म्हणजे संस्...,2


In [ ]:
train_df['Labels'].value_counts()

2    1947
0    1809
1    1737
Name: Labels, dtype: int64

In [ ]:
test_df['Labels'].value_counts()

0    442
1    438
2    379
Name: Labels, dtype: int64

In [ ]:
train_df = train_df.sample(frac = 1, random_state=seed)
train_df

,Content,Labels
1858,कुंचीतल्या कानाजवळ कुऽऽश्नऽऽ अशा माझ्या नावाचा...,1
691,मोठा झाला? जवा फर्मान आलं; तवा पोरांस्नी म्हणा...,0
4564,अंगणात आले. यति माझ्यासमोरच एका लहान शिलाखंडाव...,2
1101,केला. त्या ओसाड गावातून राजे सुर्व्यांच्या वाड...,0
5006,मी तरी माणूसच आहे ना? मी माधवला मानेनं होकार द...,2
...,...,...
3772,होत्या. मी जागा झालो आहे; की नाही; ते पाहण्याक...,2
5191,आपल्या रूपाकडं आणि वस्त्राकडं पाहू लागले; कुठं...,2
5226,पटलं ते.' 'पण मला नाही अजून पटत.' ' म्हणजे ?' ...,2
5390,struggle not availeth.' अंगाला आळेपिळे देत कुम...,2


In [ ]:
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4,
             "eval_batch_size": 32,
             "sliding_window": True}

model = ClassificationModel(
    "bert", "google/muril-base-cased",
    num_labels=len(labels),
    args=train_args
    # sliding_window=True
)

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not in

In [ ]:
model.args.learning_rate

4e-05

In [ ]:
train_df.dtypes

Content    object
Labels      int64
dtype: object

In [ ]:
model.train_model(train_df, eval_df=test_df)

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5493 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1420 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/1420 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/1420 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/1420 [00:00<?, ?it/s]

(5680, 0.09508202071994393)

In [ ]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1259 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/81 [00:00<?, ?it/s]

{'mcc': 0.9892711616382974,
 'f1': 0.9928514694201748,
 'acc': 0.9928514694201748,
 'eval_loss': 0.1535417430556809}

In [ ]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(train_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5493 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/355 [00:00<?, ?it/s]

{'mcc': 1.0, 'f1': 1.0, 'acc': 1.0, 'eval_loss': 0.0018331488200076396}

In [ ]:
wrong_predictions

[]

In [ ]:
def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [ ]:
save_model('outputs','author-classification-marathi-muril-three')

In [ ]:
!tar -zxvf ./author-classification-marathi-bert.tar.gz

tar (child): ./author-classification-marathi-bert.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
!rm -rf outputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/author-classification-marathi-muril-three.tar.gz' '/content/drive/MyDrive/Colab Notebooks'

In [ ]:
predictions, raw_outputs = model.predict([
    "आज मी फुलांची गोष्ट सांगणार आहे. लहानपणी मला फुलांचा फार नाद होता. फुलासारखी सुंदर व पवित्र वस्तू जगात कोणतीच नसेल.",
    "कारण कुणीही कधीच विसरू नये की, काळ हा अखंड आहे. जीवन ही भीक मागून मिळणारी वस्तू नव्हे. जीवन म्हणजे पूर्णत्वासाठी अनंताकडून अनंताकडे चालू असणारा अखंड प्रवास आहे!",
    "तीन पिढ्यांचा तो अतिशय बोलका मूक संवाद फक्त शांतपणे जळण्याची ताकद असलेल्या भोवतीच्या समयांनाच कळत होता!",
    "कारण द्वेषा‍तून कपट, कपटातून क्रोध आणि क्रोधातून युद्ध जन्म घेत असतं! युद्धांनी प्रश्न कधीच मिटत नसतात - उलट त्यांतून नवे जटिल प्रश्न निर्माण होतात.",
    "तू राजा असूनही राजाजवळ आवश्यक असणारी व्यक्तिपरीक्षणाची दृष्टी तुला नाही व्यक्तिपरीक्षणाची दृष्टी तुला नाही व्यक्तिपरीक्षणाची दृष्टी तुला नाही. "])

raw_outputs

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:2253: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_pred, counts = mode(pred_row)


[array([[-3.30806732, -3.19105148,  6.18804169]]),
 array([[-3.28500271,  6.4048028 , -2.32131171]]),
 array([[-3.2454896 ,  6.41652918, -2.37192917]]),
 array([[-3.45438218, -3.01245666,  6.17848158]]),
 array([[ 6.33866739, -3.3507607 , -3.427459  ]])]

In [ ]:
[labels[i] for i in predictions]

['V S Khandekar',
 'Shivaji Sawant',
 'Shivaji Sawant',
 'V S Khandekar',
 'Ranjeet Desai']

In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/author-classification-marathi-muril-three.tar.gz' '/content/'

In [ ]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('author-classification-marathi-muril-three')

In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "outputs/",
    num_labels=3,
    args=train_args,
    use_cuda=False
)

In [ ]:
validation_df = pd.read_excel('validation_dataset.xlsx')
validation_df.head()

,Content,Author
0,जिजाबाई मेण्याबाहेर आल्या. त्यांनी देवळाकडे प...,Ranjeet Desai
1,लांडग्यांचा उपद्रव थांबविण्यासाठी दादोजींनी ह...,Ranjeet Desai
2,बसक्या घराच्या कट्ट्यावर घोंगड्या पसरल्या होत्...,Ranjeet Desai
3,"खानाने खास मसलत भरविली, आणि शिवाजीची भेट घेण्...",Ranjeet Desai
4,भारावलेल्या सिद्दी हिलालने राजनिष्ठेची तलवार ...,Ranjeet Desai


In [ ]:
validation_df['Author'].value_counts()

V S Khandekar     18
Shivaji Sawant    17
Ranjeet Desai     15
Name: Author, dtype: int64

In [ ]:
labels = list(validation_df['Author'].unique())
labels.sort()
print(labels)
mapper = {labels[i]: i for i in range(len(labels))}
print(mapper)

validation_df.replace({'Author': mapper}, inplace=True)
validation_df.astype({'Author': 'int'})
validation_df.rename(columns={'Author': 'Labels'}, inplace=True)
validation_df.head()

['Ranjeet Desai', 'Shivaji Sawant', 'V S Khandekar']
{'Ranjeet Desai': 0, 'Shivaji Sawant': 1, 'V S Khandekar': 2}


,Content,Labels
0,जिजाबाई मेण्याबाहेर आल्या. त्यांनी देवळाकडे प...,0
1,लांडग्यांचा उपद्रव थांबविण्यासाठी दादोजींनी ह...,0
2,बसक्या घराच्या कट्ट्यावर घोंगड्या पसरल्या होत्...,0
3,"खानाने खास मसलत भरविली, आणि शिवाजीची भेट घेण्...",0
4,भारावलेल्या सिद्दी हिलालने राजनिष्ठेची तलवार ...,0


In [ ]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(validation_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.8619081200097488,
 'f1': 0.9,
 'acc': 0.9,
 'eval_loss': 0.9099298019500566}

In [ ]:
wrong_predictions

[{'guid': 10, 'text_a': 'युधिष्ठिर गंगेच्या प्रवाहामध्ये जाऊन गुडघाभार पाण्यात उभा होता. नदीकाठच्या एका कातळावर राजमाता कुंती बसली होती. तिच्या शेजारी द्रौपदी अधोवदन उभी होती. त्या दोघींच्या मागे तटस्थपणे कृष्ण उभा होता___ भीम, अर्जुन, नकुल, सहदेव आपल्या स्वकीयांच्या समूहात वाळूकिनाऱ्यावर व्यथित मनाने बसले होते.', 'text_b': None, 'label': 0},
 {'guid': 11, 'text_a': 'माथ्यावरचा सूर्य थोडा कलला होता. त्या सूर्यदाहात सारी भूमी होरपळत होती. पण त्या सूर्यकिरणांची तमा न बाळगता कर्ण अपराह्ण काळी आपली सूर्योपासना पुरी करीत होता. गंगेच्या पाण्यात उभा राहुन, दोन्ही हात उंचावून तो पूजामंत्र म्हणत होता. पैलतीरावर त्याची दृष्टी स्थिरावली होती. पूजारंभी आलेले सूर्यकिरण माथ्यावरून पाठीमागे परतले, तरी त्याचे भान कर्णाला नव्हते. तेजस्वी गौर वर्णाचा, सुदृढ बांध्याचा कर्ण एकाग्रपणे आपली नित्य सूर्योपासना पुरी करीत होता. रूपसंपन्न कर्णाच्या रूपाने साक्षात सूर्याची प्रतिमा गंगाजलावर प्रकटल्याचा भास होत होता.', 'text_b': None, 'label': 0},
 {'guid': 12, 'text_a': 'गंगेच्या किनाऱ्यावर भर उन्हात तळपणाऱ्या सु